In [1]:
%load_ext autoreload
%autoreload 2

# Helpers

In [2]:
import re
from collections import Counter, defaultdict
import itertools


In [3]:
def id(x): return x

def read(day, part=1, process=id):
    with open(f'./inputs/{day}.{part}.txt') as f:
        return list(map(process, map(str.strip, f)))
    
def partition_into(lst: list, when, preprocess=id, postprocess=id) -> list[list]:
    res = [[], ]
    for item in map(preprocess, lst):
        if when(item): res.append([])
        else: res[-1].append(postprocess(item))
    return res

# Day 1

In [4]:
input = partition_into(read(1), lambda x: not x, postprocess=int)
calories = [sum(l) for  l in input]
calories.sort()
print(calories[-1])

66487


In [5]:
print(sum(calories[-3:]))

197301


# Day 2

In [6]:
scores = {
    'A X': 1+3,
    'A Y': 2+6,
    'A Z': 3+0,
    'B X': 1+0,
    'B Y': 2+3,
    'B Z': 3+6,
    'C X': 1+6,
    'C Y': 2+0,
    'C Z': 3+3,
}

In [7]:
input = read(2, 1)

In [8]:
print(sum(map(scores.get, input)))

10994


In [9]:
scores2 = {
    'A X': 3+0,
    'A Y': 1+3,
    'A Z': 2+6,
    'B X': 1+0,
    'B Y': 2+3,
    'B Z': 3+6,
    'C X': 2+0,
    'C Y': 3+3,
    'C Z': 1+6,
}

In [10]:
print(sum(map(scores2.get, input)))

12526


# Day 3

In [11]:
import string

priority = dict(enumerate(string.ascii_lowercase, 1))
priority |= dict(enumerate(string.ascii_uppercase, 27))
priority = {v:k for k,v in priority.items()}

In [12]:
input = read(3, 1)

In [13]:
def split_half (x):
    n = len(x)//2
    return set(x[:n]), set(x[n:])

common = [x&y for x, y in map(split_half, input)]

In [14]:
sum(sum(map(priority.get, c)) for c in common)

7848

In [15]:
groups = [input[i:i+3] for i in range(0, len(input), 3)]
groups = [list(set(x)&set(y)&set(z))[0] for x, y, z in groups]
sum(map(priority.get, groups))

2616

# Day 4